# McCall Model

Pablo Winant

### Job-Search Model

-   When unemployed in date, a job-seeker
    -   consumes unemployment benefit $c_t = \underline{c}$
    -   receives in every date $t$ a job offer $w_t$
        -   $w_t$ is i.i.d.,
        -   takes values $w_1, w_2, w_3$ with probabilities
            $p_1, p_2, p_3$
    -   if job-seeker accepts, becomes employed at rate $w_t$ in the
        next period
    -   else he stays unemployed
-   When employed at rate $w$
    -   worker consumes salary $c_t = w$
    -   with small probability $\lambda>0$ looses his job:
        -   starts next period unemployed
    -   otherwise stays employed at same rate
-   Objective: $\max E_0 \left\{ \sum \beta^t \log(w_t) \right\}$

**What are the states, the controls, the reward of this problem ? Write
down the Bellman equation.**

The states are: - unemployed, $w=w_1$ - unemployed, $w=w_2$ -
unemployed, $w=w_3$ - employed, $w=w_1$ - employed, $w=w_2$ - employed,
$w=w_3$

The state-space can also be represented as
$(employed,unemployed)\times (w_1, w_2, w_3)$

The controls are $accept/reject$ when unemployed, the empty set
otherwise.

The reward is $U(\underbar{c})$ if unemployed, $U(\underbar{w})$ if
employed at wage $w$.

**Define a parameter structure for the model.**

In [61]:
m = (;
    wages=[0.9, 1.0, 1.1],
    probas=[1/3, 1/3, 1/3],
    β = 0.9,
    λ = 0.01,
    cbar = 0.8,
    punishment=0.0
)
U(x) = log(x)

U (generic function with 1 method)

In [14]:
V_U_0 = [0.0,0.0,0.0]
V_E_0 = [0.0,0.0,0.0]
x_0 = [false, false, false]

3-element Vector{Bool}:
 0
 0
 0

**Define a function
`value_update(V_U::Vector{Float64}, V_E::Vector{Float64}, x::Vector{Bool}, p::Parameters)::Tuple{Vector, Vector}`,
which takes in value functions tomorrow and a policy vector and return
updated values for today.**

In [62]:
function value_update(V_U, V_E, x, m)

    # compute the continuation value of being unemployed tomorrow
    
    (;λ, β, cbar, wages, probas, punishment) = m

    cont_V_U = sum(   probas[i]*V_U[i]  for i=1:3   )

    n_V_E = zeros(3)
    for i=1:3
        n_V_E[i] = U(wages[i]) + β*(1-λ)*V_E[i] + β*λ*cont_V_U
    end

    n_V_U = zeros(3)
    for i=1:3
        if x[i] # if accept
            n_V_U[i] = U(cbar) + β*V_E[i]
        else # if reject
            n_V_U[i] = U(cbar) + β*cont_V_U - punishment
        end

    end

    # vector version
    # n_V_E = U.(wages) + β*(1-λ)*V_E

    return n_V_U, n_V_E

end

value_update(V_U_0, V_E_0, x_0, m)

([-0.2231435513142097, -0.2231435513142097, -0.2231435513142097], [-0.10536051565782628, 0.0, 0.09531017980432493])

**Define a function
`policy_eval(x::Vector{Bool}, p::Parameter)::Tuple{Vector, Vector}`
which takes in a policy vector and returns the value(s) of following
this policies forever. You can add relevant arguments to the function.**

In [63]:
distance(v1::Vector, v2::Vector) = maximum(abs.(v2-v1))

distance (generic function with 1 method)

In [64]:
function policy_eval(x, m; T=1000, τ_η=1e-10, verbose=false)

    V_U_0 = [0.0,0.0,0.0]
    V_E_0 = [0.0,0.0,0.0]

    for t=1:T
        V_U_1, V_E_1 = value_update(V_U_0, V_E_0, x, m)
        η = distance( [V_U_0; V_E_0], [V_U_1; V_E_1] )

        V_U_0, V_E_0 = V_U_1, V_E_1

        if verbose

            println("Iteration $(t): $(η)")
        end

        if η<τ_η
            return V_U_0, V_E_0
        end

    end

    error("No convergence")

    
end

policy_eval (generic function with 1 method)

In [65]:
policy_eval(x, m; T=1000, verbose=true);

Iteration 1: 0.2231435513142097
Iteration 2: 0.20082919618278872
Iteration 3: 0.18074627656450992
Iteration 4: 0.16267164890805885
Iteration 5: 0.14640448401725292
Iteration 6: 0.13176403561552763
Iteration 7: 0.11858763205397493
Iteration 8: 0.10672886884857746
Iteration 9: 0.09605598196371967
Iteration 10: 0.08645038376734782
Iteration 11: 0.0778053453906129
Iteration 12: 0.07002481085155154
Iteration 13: 0.06302232976639632
Iteration 14: 0.05672009678975676
Iteration 15: 0.051048087110781015
Iteration 16: 0.045943278399702914
Iteration 17: 0.041348950559732645
Iteration 18: 0.03721405550375967
Iteration 19: 0.033492649953383635
Iteration 20: 0.03014338495804525
Iteration 21: 0.02712904646224068
Iteration 22: 0.0244161418160167
Iteration 23: 0.021974527634414542
Iteration 24: 0.019777074870973355
Iteration 25: 0.01779936738387633
Iteration 26: 0.016019430645488608
Iteration 27: 0.014417487580939525
Iteration 28: 0.012975738822845528
Iteration 29: 0.011678164940561508
Iteration 30: 0.

**Define a function
`bellman_step(V_E::Vector, V_U::Vector, p::Parameters)::Tuple{Vector, Vector, Vector}`
which returns updated values, together with improved policy rules.**

In [66]:
[true, false]

2-element Vector{Bool}:
 1
 0

In [67]:
zeros(Bool, 3)

3-element Vector{Bool}:
 0
 0
 0

In [68]:
function bellman_step(V_U, V_E, m;)

    # compute the continuation value of being unemployed tomorrow
    
    (;λ, β, cbar, wages, probas, punishment) = m

    cont_V_U = sum(   probas[i]*V_U[i]  for i=1:3   )

    n_V_E = zeros(3)
    for i=1:3
        n_V_E[i] = U(wages[i]) + β*(1-λ)*V_E[i] + β*λ*cont_V_U
    end

    n_V_U = zeros(3)
    x = zeros(Bool, 3)
    for i=1:3
        v_accept = U(cbar) + β*V_E[i]
        v_reject = U(cbar) + β*cont_V_U
        if v_accept>v_reject
            n_V_U[i] = v_accept
            x[i] = true
        else v_accept>v_reject
            n_V_U[i] = v_reject - punishment
            x[i] = false
        end
    end

    # vector version
    # n_V_E = U.(wages) + β*(1-λ)*V_E

    return n_V_U, n_V_E, x

end

bellman_step (generic function with 1 method)

In [69]:
bellman_step(V_U_0, V_E_0, m);

**Implement Value Function**

In [70]:
function value_function_iteration(m; T=1000, τ_η=1e-10, verbose=false)

    V_U_0 = [0.0,0.0,0.0]
    V_E_0 = [0.0,0.0,0.0]

    for t=1:T

        V_U_1, V_E_1, x = bellman_step(V_U_0, V_E_0, m)
        η = distance( [V_U_0; V_E_0], [V_U_1; V_E_1] )

        V_U_0, V_E_0 = V_U_1, V_E_1

        if verbose
            

            println("Iteration $(t): $(η) : $(x)")
        end

        if η<τ_η
            return V_U_0, V_E_0, x
        end

    end

    error("No convergence")

    
end

value_function_iteration (generic function with 1 method)

In [48]:
value_function_iteration(m, verbose=true)

Iteration 1: 0.2231435513142097 : Bool[0, 0, 0]
Iteration 2: 0.09588451141295112 : Bool[1, 1, 1]
Iteration 3: 0.08629606027165598 : Bool[1, 1, 1]
Iteration 4: 0.07618551515584138 : Bool[0, 1, 1]
Iteration 5: 0.06776620825824747 : Bool[0, 1, 1]
Iteration 6: 0.06017209711438726 : Bool[0, 1, 1]
Iteration 7: 0.05339623450323355 : Bool[0, 1, 1]
Iteration 8: 0.04737184303259734 : Bool[0, 1, 1]
Iteration 9: 0.042021967211819056 : Bool[0, 1, 1]
Iteration 10: 0.037273116454005106 : Bool[0, 1, 1]
Iteration 11: 0.033058519006658904 : Bool[0, 1, 1]
Iteration 12: 0.029318432293365393 : Bool[0, 1, 1]
Iteration 13: 0.025999667465517406 : Bool[0, 1, 1]
Iteration 14: 0.023054948225977645 : Bool[0, 1, 1]
Iteration 15: 0.020442277375651963 : Bool[0, 1, 1]
Iteration 16: 0.018124355344447007 : Bool[0, 1, 1]
Iteration 17: 0.01606805806256495 : Bool[0, 1, 1]
Iteration 18: 0.014243971400986988 : Bool[0, 1, 1]
Iteration 19: 0.01262597700763557 : Bool[0, 0, 1]
Iteration 20: 0.011258315445731748 : Bool[0, 0, 1]


([-0.12917371935610356, -0.12917371935610356, 0.5715802122005743], [-0.9579891504189799, 0.00862108482895072, 0.883026404003542], Bool[0, 0, 1])

**Implement Policy Iteration and compare rates of convergence.**

In [52]:
function policy_iteration(m; T=1000, τ_η=1e-10, verbose=false)

    V_U_0 = [0.0,0.0,0.0]
    V_E_0 = [0.0,0.0,0.0]

    x0 = [false, false, false]

    for t=1:T

        V_U_0, V_E_0 = policy_eval(x0, m)    
    
        V_U_1, V_E_1, x = bellman_step(V_U_0, V_E_0, m)

        η = maximum(abs.(x-x0))

        x0 = x

        if verbose
            
            println("Iteration $(t): $(η) : $(x)")
        end

        if η<τ_η
            return V_U_0, V_E_0, x
        end

    end

    error("No convergence")

    
end

policy_iteration (generic function with 1 method)

In [54]:
policy_iteration(m; verbose=true)

Iteration 1: 1 : Bool[1, 1, 1]
Iteration 2: 1 : Bool[0, 1, 1]
Iteration 3: 1 : Bool[0, 0, 1]
Iteration 4: 0 : Bool[0, 0, 1]

([-0.12917371930468718, -0.12917371930468718, 0.5715802122519905], [-0.95798915039766, 0.008621084866072553, 0.8830264040549584], Bool[0, 0, 1])

**Discuss the Effects of the Parameters**

With the baseline calibration workers accept only the highest offer.
What can make them accept more offer?

In [60]:
policy_iteration(merge(m, (;cbar=0.5)))[3]

3-element Vector{Bool}:
 0
 1
 1

In [75]:
policy_iteration(merge(m, (;punishment=0.2)))[3]

3-element Vector{Bool}:
 0
 1
 1

In [78]:
# higher unemployment risk
policy_iteration(merge(m, (;λ=0.1)))[3]

3-element Vector{Bool}:
 0
 1
 1

In [81]:
# higher unemployment risk
policy_iteration(merge(m, (;β=0.8)))[3]

3-element Vector{Bool}:
 0
 1
 1